In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import *

In [3]:
from pyspark import Broadcast

In [4]:
from pyspark.sql.types import *

In [6]:
from pyspark.sql.window import Window

In [5]:
spark = SparkSession.builder.appName("new saprk session").getOrCreate()

25/03/25 12:02:11 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 10.1.209.120 instead (on interface wlp0s20f3)
25/03/25 12:02:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 12:02:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/25 12:02:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/25 12:02:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [20]:
data = [("helo uhdjasdj jvjsj jdvnsj",),("hbsh dbvsabj bdshabs",)]
df = spark.createDataFrame(data,["columns"])

In [21]:
df.show()


+--------------------+
|             columns|
+--------------------+
|helo uhdjasdj jvj...|
|hbsh dbvsabj bdshabs|
+--------------------+



In [28]:
df.withColumn("columns",initcap(col("columns"))).show()

+--------------------+
|             columns|
+--------------------+
|Helo Uhdjasdj Jvj...|
|Hbsh Dbvsabj Bdshabs|
+--------------------+



In [16]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

""" Converting function to UDF """
convertUDF = udf(lambda z: convertCase(z))

df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
.show(truncate=False)

def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z),StringType())    

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
.show(truncate=False)

""" Using UDF on SQL """
spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)
     
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " + \
          "where Name is not null and convertUDF(Name) like '%John%'") \
     .show(truncate=False)  
     
""" null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)  

25/03/25 12:15:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+

+-----+-----------+
|Seqno|Name       |
+-----+-----------+
|1    |John Jones |
+-----+-----------+

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |NULL        |
+-----+------------+

+------------------+
|_nul

In [29]:
emp = [(1,"hbdhs","01"),
      (2,"jdcj","02"),
      (3,"iiac","01")]
dept = [("01","IT"),
       ("02","HR"),
       ("03","Enc")]
emp_df = spark.createDataFrame(data =emp,schema=["id","name","dept_code"])
dept_df = spark.createDataFrame(data = dept,schema = ["dept_code","dept"])

joined_df = emp_df.join(broadcast(dept_df),"dept_code","leftsemi")

In [31]:
joined_df.show()

+---------+---+-----+
|dept_code| id| name|
+---------+---+-----+
|       01|  1|hbdhs|
|       02|  2| jdcj|
|       01|  3| iiac|
+---------+---+-----+



In [33]:
joined_df2 = dept_df.join(broadcast(emp_df),"dept_code","leftanti")

In [34]:
joined_df2.show()

+---------+----+
|dept_code|dept|
+---------+----+
|       03| Enc|
+---------+----+

